In [1]:
import os

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
%pwd

'c:\\Users\\jason\\Desktop\\steam-review-generator\\notebooks'

In [4]:
os.chdir('../')

In [5]:
from steamReviewGenerator.constants import *
from steamReviewGenerator.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        print(config_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import urllib.request as request
import zipfile
from steamReviewGenerator import logger
from steamReviewGenerator.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'Downloaded {filename} with the following info: \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

config\config.yaml
[2023-08-16 15:43:14,544: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-16 15:43:14,545: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-16 15:43:14,546: INFO: common: Created Directory At: artifacts]
[2023-08-16 15:43:14,547: INFO: common: Created Directory At: artifacts/data_ingestion]
[2023-08-16 15:43:16,812: INFO: 1805441797: Downloaded artifacts/data_ingestion/battlebit-remastered-reviews.zip with the following info: 
Connection: close
Content-Length: 5430412
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2706ac68aafa9b1b5e23595bd35a3d892af8f1cd59485b2b5f0379bd58e68b0b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D856:6B00:A5B209:C0D16A:64DD34E1
Accept-Ranges: bytes
Date: Wed, 16 Aug 2023 20:43:15 GMT
Via: 1.1 varn